In [ ]:
%matplotlib inline
import math
import re
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import defaultdict
from scipy.stats import pearsonr
import pandas as pd
from IPython.display import display


In [ ]:
DATA_DIR = r"Y:\Studium\3. Sem UniPI\Data Analytics 4 digital Health\Data"

DATASETS = {
    "heart_diagnoses_1": "heart_diagnoses_1.csv",
    "laboratory_events_codes_2": "laboratory_events_codes_2.csv",
    "microbiology_events_codes_3": "microbiology_events_codes_3.csv",
    "procedure_code_4": "procedure_code_4.csv",
}
name = "microbiology_events_codes_3"

In [ ]:
df = pd.read_csv(f"{DATA_DIR}/{DATASETS['laboratory_events_codes_2']}", index_col=False)

df.columns

# INspections

## A

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

## B

In [ ]:
df.columns

In [ ]:
df[df.columns[:10]].head(10)

In [ ]:
df[df.columns[10:]].head(10)

## C

In [ ]:
df.columns

In [ ]:
df.info()

# Data Understanding and Preprocessing, cleaning of DF1 - heart

In [ ]:
df.duplicated().sum()

In [ ]:
#show duplicated rows
df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()).head(20)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
for col in df.columns:
    if col == 'charttime' or col == 'hadm_id' or col == 'subject_id':
        continue
    print(f"{col}: {df[col].value_counts()}")

## Check for wrong NaNs / non typical entries in each column

#### Find wrong NaNs

In [ ]:
print("="*80)
print("FIND NON-NUMERICAL ENTRIES IN NUMERICAL COLUMNS")
print("="*80)

# Identify columns that should be numerical
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nColumns already numeric: {len(numeric_cols)}")
print(numeric_cols)

# Check object/string columns that might contain numerical data
object_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nObject/String columns to check: {len(object_cols)}")
print(object_cols)

# For each object column, try to find non-numerical entries
print("\n" + "="*80)
print("CHECKING FOR NON-NUMERICAL ENTRIES")
print("="*80)

non_numerical_summary = {}

for col in object_cols:
    non_numerical_entries = []
    
    for idx, value in df[col].items():
        if pd.isna(value):  # Skip NaN/None
            continue
        
        # Try to convert to float
        try:
            float(value)
        except (ValueError, TypeError):
            non_numerical_entries.append({
                'index': idx,
                'value': value,
                'type': type(value).__name__
            })
    
    # Store summary
    if non_numerical_entries:
        non_numerical_summary[col] = non_numerical_entries
        
        print(f"\n{'─'*80}")
        print(f"Column: '{col}' | Non-numerical entries: {len(non_numerical_entries)}")
        print(f"{'─'*80}")
        
        # Get unique non-numerical values
        unique_values = list(set([e['value'] for e in non_numerical_entries]))
        print(f"Unique non-numerical values ({len(unique_values)}):")
        for val in sorted(unique_values)[:20]:  # Show first 20
            count = sum(1 for e in non_numerical_entries if e['value'] == val)
            print(f"  • '{val}' — appears {count} times")
        
        if len(unique_values) > 20:
            print(f"  ... and {len(unique_values) - 20} more")
        
        # Show sample rows
        print(f"\nSample rows with non-numerical entries:")
        for entry in non_numerical_entries[:5]:
            print(f"  Index {entry['index']}: {entry['value']!r} ({entry['type']})")

# Summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nColumns with non-numerical entries: {len(non_numerical_summary)}")

for col, entries in non_numerical_summary.items():
    total_rows = len(df)
    non_num_count = len(entries)
    pct = (non_num_count / total_rows) * 100
    unique_count = len(set([e['value'] for e in entries]))
    
    print(f"\n{col}:")
    print(f"  Non-numerical rows: {non_num_count} ({pct:.2f}%)")
    print(f"  Unique non-numerical values: {unique_count}")
    print(f"  Numerical rows: {total_rows - non_num_count}")

# Optional: Create a detailed report
if non_numerical_summary:
    print("\n" + "="*80)
    print("DETAILED REPORT - ALL NON-NUMERICAL ENTRIES")
    print("="*80)
    
    for col in non_numerical_summary:
        print(f"\n{col}:")
        entries_df = pd.DataFrame(non_numerical_summary[col])
        # Count occurrences
        value_counts = entries_df['value'].value_counts()
        print(value_counts)

- valueuom: has '' , remove; Pos/Neg == +/-; U ??
- value has wrong entries inspect and extrat if possible to valuenum


### Handle Value wrong nans, then extract missing from value into new column valuenum_merged if possible

In [ ]:
# # chec k only value column: show all unique non-numerical entries in 'value' column
# non_numerical_values = []
# for idx, value in df['value'].items():
#     if pd.isna(value):  # Skip NaN/None
#         continue
    
#     # Try to convert to float
#     try:
#         float(value)
#     except (ValueError, TypeError):
#         non_numerical_values.append({
#             'index': idx,
#             'value': value,
#             'type': type(value).__name__
#         })
# print(f"\n{'─'*80}")
# print(f"Column: 'value' | Non-numerical entries: {len(non_numerical_values)}")
# print(f"{'─'*80}")
# # Get unique non-numerical values
# unique_values = list(set([e['value'] for e in non_numerical_values]))
# print(f"Unique non-numerical values ({len(unique_values)}):")
# for val in sorted(unique_values):
#     count = sum(1 for e in non_numerical_values if e['value'] == val)
#     print(f"  • '{val}' — appears {count} times")   
    

FINDINGS

=> in col value, We want to convert '___' and 'NONE' 'ERROR' to np.nan!

=> then we create a new col value_extracted (float64) out of col value where:
- we can calculate as float complete / like 20/0 but only if there is anumber before and after the /! => complete
- we can take the middle point of complete ranges like '80-160'
- we can calculate a float value of comparisons with < > by sub/add 0.1 to the number, eg. '>1.050' => 1.150 or '<1' => 0.9
- the rest is to nuemic error coerce put to NaN.

=> then, we fill np.nan entries in col valuenum with values from valUe_extracted if they are not nan and tell me the amount of filled rows and show examples beffore and after

In [ ]:
#first sanity check if needed
# check are there rows where valuenum has nan but value has a value?a
len(df[(df['valuenum'].isna()) & (df['value'].notna())])

In [ ]:
print("="*80)
print("STEP 1: CONVERT PLACEHOLDER STRINGS TO NaN")
print("="*80)

# Define placeholder patterns
placeholders = ['___', 'NONE', 'ERROR']

print(f"\nPlaceholders to convert: {placeholders}")
print(f"Before: {df['value'].isna().sum()} NaN values")

# Convert placeholders to NaN (case-insensitive)
for placeholder in placeholders:
    mask = df['value'].astype(str).str.lower() == placeholder.lower()
    count = mask.sum()
    df.loc[mask, 'value'] = np.nan
    print(f"  Converted '{placeholder}': {count} rows")

print(f"After: {df['value'].isna().sum()} NaN values")

In [ ]:
print("="*80)
print("STEP 2: EXTRACT NUMERIC VALUES FROM 'value' COLUMN")
print("="*80)

def extract_numeric_from_value(x):
    """
    Extract numeric values from various string formats:
    - Simple numbers: '123' => 123
    - Decimals: '123.45' => 123.45
    - Divisions: '20/10' => 2.0
    - Ranges: '80-160' => 120 (midpoint)
    - Comparisons: '>1.050' => 1.150, '<1' => 0.9
    """
    
    if pd.isna(x):
        return np.nan
    
    x_str = str(x).strip()
    
    # Try direct float conversion
    try:
        return float(x_str)
    except ValueError:
        pass
    
    # Handle divisions (e.g., '20/10')
    if '/' in x_str:
        parts = x_str.split('/')
        try:
            if len(parts) == 2 and parts[0].strip() and parts[1].strip():
                num1 = float(parts[0].strip())
                num2 = float(parts[1].strip())
                if num2 != 0:  # Avoid division by zero
                    return num1 / num2
        except (ValueError, ZeroDivisionError):
            pass
    
    # Handle ranges (e.g., '80-160')
    if '-' in x_str and not x_str.startswith('-'):
        parts = x_str.split('-')
        try:
            if len(parts) == 2 and parts[0].strip() and parts[1].strip():
                num1 = float(parts[0].strip())
                num2 = float(parts[1].strip())
                return (num1 + num2) / 2  # Midpoint
        except ValueError:
            pass
    
    # Handle comparisons (e.g., '>1.050' => 1.150, '<1' => 0.9)
    comparison_match = re.match(r'^([<>]=?)(\d*\.?\d+)$', x_str.strip())
    if comparison_match:
        operator = comparison_match.group(1)
        try:
            num = float(comparison_match.group(2))
            if operator == '>':
                return num + 0.1
            elif operator == '>=':
                return num
            elif operator == '<':
                return num - 0.1
            elif operator == '<=':
                return num
        except ValueError:
            pass
    
    # If nothing worked, return NaN
    return np.nan

# Apply extraction
df['value_extracted'] = df['value'].apply(extract_numeric_from_value)

print(f"\nExtraction complete!")
print(f"Non-null values extracted: {df['value_extracted'].notna().sum():,}")
print(f"Failed extractions (NaN): {df['value_extracted'].isna().sum():,}")

In [ ]:
# print("="*80)
# print("STEP 3: SHOW EXAMPLES OF EXTRACTED VALUES (Complex Formats Only)")
# print("="*80)

# # Find rows with successful extractions
# extracted_rows = df[df['value_extracted'].notna() & df['value'].notna()].copy()

# print(f"\nTotal rows with extracted values: {len(extracted_rows):,}\n")

# # ────────────────────────────────────────────────────────────────────────────
# # COMPARISONS: >, <, >=, <=
# # ────────────────────────────────────────────────────────────────────────────
# print("="*80)
# print("EXAMPLES: COMPARISONS (>, <, >=, <=)")
# print("="*80)

# comparison_pattern = r'^([<>]=?)(\d*\.?\d+)$'
# comparison_examples = []

# for idx, row in extracted_rows.iterrows():
#     original = str(row['value']).strip()
#     if re.match(comparison_pattern, original):
#         comparison_examples.append({
#             'original': original,
#             'extracted': row['value_extracted']
#         })

# if comparison_examples:
#     # Show first 15 unique examples
#     unique_comparisons = []
#     seen = set()
#     for ex in comparison_examples:
#         if ex['original'] not in seen:
#             unique_comparisons.append(ex)
#             seen.add(ex['original'])
#             if len(unique_comparisons) >= 15:
#                 break
    
#     print(f"\nFound {len(comparison_examples)} comparison values\n")
#     print(f"{'Original':<20} {'Extracted':<15} {'Logic':<40}")
#     print("─" * 75)
    
#     for ex in unique_comparisons:
#         original = ex['original']
#         extracted = ex['extracted']
        
#         # Explain the logic
#         if original.startswith('>'):
#             if original.startswith('>='):
#                 logic = f"'{original}' → value as-is"
#             else:
#                 num = float(original[1:])
#                 logic = f"'{original}' → {num} + 0.1 = {extracted}"
#         elif original.startswith('<'):
#             if original.startswith('<='):
#                 logic = f"'{original}' → value as-is"
#             else:
#                 num = float(original[1:])
#                 logic = f"'{original}' → {num} - 0.1 = {extracted}"
        
#         print(f"{original:<20} {extracted:<15.3f} {logic:<40}")
# else:
#     print("\nNo comparison values found")

# # ────────────────────────────────────────────────────────────────────────────
# # RANGES: 80-160, 0-2, etc.
# # ────────────────────────────────────────────────────────────────────────────
# print("\n" + "="*80)
# print("EXAMPLES: RANGES (e.g., 80-160, 0-2)")
# print("="*80)

# range_pattern = r'^(\d+\.?\d*)-(\d+\.?\d*)$'
# range_examples = []

# for idx, row in extracted_rows.iterrows():
#     original = str(row['value']).strip()
#     if re.match(range_pattern, original) and not original.startswith('-'):
#         range_examples.append({
#             'original': original,
#             'extracted': row['value_extracted']
#         })

# if range_examples:
#     # Show first 15 unique examples
#     unique_ranges = []
#     seen = set()
#     for ex in range_examples:
#         if ex['original'] not in seen:
#             unique_ranges.append(ex)
#             seen.add(ex['original'])
#             if len(unique_ranges) >= 15:
#                 break
    
#     print(f"\nFound {len(range_examples)} range values\n")
#     print(f"{'Original':<20} {'Extracted':<15} {'Logic (Midpoint)':<40}")
#     print("─" * 75)
    
#     for ex in unique_ranges:
#         original = ex['original']
#         extracted = ex['extracted']
        
#         # Calculate and show logic
#         parts = original.split('-')
#         num1 = float(parts[0])
#         num2 = float(parts[1])
#         midpoint = (num1 + num2) / 2
        
#         logic = f"({num1} + {num2}) / 2 = {midpoint}"
        
#         print(f"{original:<20} {extracted:<15.3f} {logic:<40}")
# else:
#     print("\nNo range values found")

# # ────────────────────────────────────────────────────────────────────────────
# # DIVISIONS: 20/10, 15/3, etc.
# # ────────────────────────────────────────────────────────────────────────────
# print("\n" + "="*80)
# print("EXAMPLES: DIVISIONS (e.g., 20/10, 15/3)")
# print("="*80)

# division_pattern = r'^(\d+\.?\d*)/(\d+\.?\d*)$'
# division_examples = []

# for idx, row in extracted_rows.iterrows():
#     original = str(row['value']).strip()
#     if re.match(division_pattern, original):
#         division_examples.append({
#             'original': original,
#             'extracted': row['value_extracted']
#         })

# if division_examples:
#     # Show first 15 unique examples
#     unique_divisions = []
#     seen = set()
#     for ex in division_examples:
#         if ex['original'] not in seen:
#             unique_divisions.append(ex)
#             seen.add(ex['original'])
#             if len(unique_divisions) >= 15:
#                 break
    
#     print(f"\nFound {len(division_examples)} division values\n")
#     print(f"{'Original':<20} {'Extracted':<15} {'Logic':<40}")
#     print("─" * 75)
    
#     for ex in unique_divisions:
#         original = ex['original']
#         extracted = ex['extracted']
        
#         # Calculate and show logic
#         parts = original.split('/')
#         num1 = float(parts[0])
#         num2 = float(parts[1])
#         result = num1 / num2 if num2 != 0 else float('nan')
        
#         logic = f"{num1} / {num2} = {result}"
        
#         print(f"{original:<20} {extracted:<15.3f} {logic:<40}")
# else:
#     print("\nNo division values found")

# # ────────────────────────────────────────────────────────────────────────────
# # SUMMARY STATISTICS
# # ────────────────────────────────────────────────────────────────────────────
# print("\n" + "="*80)
# print("EXTRACTION SUMMARY")
# print("="*80)

# print(f"\nTotal extracted values: {len(extracted_rows):,}")
# print(f"  • Comparisons:  {len(comparison_examples):,} ({len(comparison_examples)/len(extracted_rows)*100:.1f}%)")
# print(f"  • Ranges:       {len(range_examples):,} ({len(range_examples)/len(extracted_rows)*100:.1f}%)")
# print(f"  • Divisions:    {len(division_examples):,} ({len(division_examples)/len(extracted_rows)*100:.1f}%)")
# print(f"  • Other:        {len(extracted_rows) - len(comparison_examples) - len(range_examples) - len(division_examples):,}")

# print(f"\n{'─'*80}")
# print("STATISTICS OF EXTRACTED VALUES")
# print(f"{'─'*80}")
# print(df['value_extracted'].describe())

In [ ]:
print("="*80)
print("STEP 4: MERGE valuenum + value_extracted")
print("="*80)

# Store counts before merge
valuenum_before = df['valuenum'].notna().sum()
value_extracted_only = (df['valuenum'].isna() & df['value_extracted'].notna()).sum()

print(f"\nBefore merge:")
print(f"  valuenum (non-null):           {valuenum_before:,}")
print(f"  value_extracted (non-null):    {df['value_extracted'].notna().sum():,}")
print(f"  Can be filled from extraction:  {value_extracted_only:,}")

# Merge: prefer valuenum, fallback to value_extracted
df['valuenum_merged'] = df['valuenum'].fillna(df['value_extracted'])

valuenum_after = df['valuenum_merged'].notna().sum()
newly_filled = valuenum_after - valuenum_before

print(f"\nAfter merge:")
print(f"  valuenum_merged (non-null):   {valuenum_after:,}")
print(f"  Newly filled from extraction:   {newly_filled:,} (+{(newly_filled/len(df)*100):.2f}%)")
print(f"  Total improvement:              {valuenum_after - df['valuenum'].isna().sum():,} rows")

print(f"\nData type: {df['valuenum_merged'].dtype}")
print(f"\nStatistics:")
print(df['valuenum_merged'].describe())

In [ ]:
# print("="*80)
# print("STEP 5: SHOW BEFORE/AFTER EXAMPLES")
# print("="*80)

# # Find rows that were newly filled
# newly_filled_rows = df[(df['valuenum'].isna()) & (df['value_extracted'].notna())].copy()

# print(f"\nTotal newly filled rows: {len(newly_filled_rows):,}\n")
# print("BEFORE → AFTER Examples:\n")
# print(f"{'Original value':<20} {'Extracted':<15} {'valuenum (before)':<20} {'valuenum_merged (after)':<20}")
# print("─" * 75)

# for i, (idx, row) in enumerate(newly_filled_rows.head(20).iterrows()):
#     original = str(row['value'])[:19]
#     extracted = f"{row['value_extracted']:.3f}" if pd.notna(row['value_extracted']) else "NaN"
#     before = "NaN"
#     after = f"{row['valuenum_merged']:.3f}"
    
#     print(f"{original:<20} {extracted:<15} {before:<20} {after:<20}")

# print(f"\n{'─'*75}")
# print(f"\nCoverage improvement:")
# print(f"  Before: {(df['valuenum'].notna().sum() / len(df) * 100):.2f}% coverage")
# print(f"  After:  {(df['valuenum_merged'].notna().sum() / len(df) * 100):.2f}% coverage")
# print(f"  Gain:   {(newly_filled / len(df) * 100):.2f}%")

In [ ]:
df.info()

 ## Convert datetimes

In [ ]:
cols = ['charttime',]  

for col in cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"{col}: parsed {df[col].notna().sum()} values, {df[col].isna().sum()} NaT")

display(df[[c for c in df.columns if c in cols]])

## inspect qc_flag == FAIL

In [ ]:
# check qc_flag == 'FAIL' 
df["qc_flag"].value_counts(), 19570/ len(df), 78124 / len(df)

2% fail, 8% warn.

IDEA: small percentage => set valuenum_merged to np.nan those rows bc qualtiy control failed

In [ ]:
# Handle QC flags
qc_fail_mask = df['qc_flag'] == 'FAIL'
qc_warn_mask = df['qc_flag'] == 'WARN'

# Drop failed measurements from valuenum_merged
before_non_null = df['valuenum_merged'].notna().sum()
df.loc[qc_fail_mask, 'valuenum_merged'] = np.nan
after_non_null = df['valuenum_merged'].notna().sum()

print(f"set {before_non_null - after_non_null:,} FAIL measurements from valuenum_merged to nan.")
print(f"Coverage drop: {(before_non_null - after_non_null) / len(df) * 100:.2f}%")

# Binary QC features for downstream aggregation/clustering
df['is_qc_fail'] = qc_fail_mask.astype(int)
df['is_qc_warn'] = qc_warn_mask.astype(int)
df['is_qc_ok'] = (~qc_fail_mask & ~qc_warn_mask).astype(int)

print(df[['qc_flag', 'is_qc_ok', 'is_qc_warn', 'is_qc_fail']].head())

## Little intermed inspection 

In [ ]:
df.info()

In [ ]:
df[df.columns].head(10)

## check if flag indidactor (abnormal) is correect (valuenum_merged is within the ranges)

In [ ]:
# convert nan / none null vals in flag to 'normal'
df['flag'] = df['flag'].fillna('normal')

In [ ]:
df["flag"].value_counts()

In [ ]:
# Check if 'flag' correctly indicates abnormal values based on reference ranges
print("="*80)
print("FLAG CORRECTNESS VALIDATION")
print("="*80)

# Create working columns
df['_val'] = pd.to_numeric(df['valuenum_merged'], errors='coerce')
df['_ref_low'] = pd.to_numeric(df['ref_range_lower'], errors='coerce')
df['_ref_high'] = pd.to_numeric(df['ref_range_upper'], errors='coerce')

# Compute expected flag based on reference ranges
def compute_expected_flag(row):
    val = row['_val']
    low = row['_ref_low']
    high = row['_ref_high']
    
    if pd.isna(val):
        return np.nan  # Can't determine without a value
    if pd.isna(low) and pd.isna(high):
        return np.nan  # No reference range available
    
    # Check if outside bounds
    if pd.notna(low) and pd.notna(high):
        return 'abnormal' if (val < low or val > high) else 'normal'
    elif pd.notna(low):
        return 'abnormal' if val < low else 'normal'
    elif pd.notna(high):
        return 'abnormal' if val > high else 'normal'
    return np.nan

df['_expected_flag'] = df.apply(compute_expected_flag, axis=1)

# Compare with actual flag
checkable = df[df['_expected_flag'].notna()].copy()
checkable['_match'] = checkable['flag'] == checkable['_expected_flag']

# Identify mismatches
mismatches = checkable[~checkable['_match']]

# Summary
print(f"\nTotal rows: {len(df):,}")
print(f"Rows with computable expected flag: {len(checkable):,}")
print(f"Matching flags: {checkable['_match'].sum():,} ({checkable['_match'].mean()*100:.2f}%)")
print(f"Mismatched flags: {len(mismatches):,} ({len(mismatches)/len(checkable)*100:.2f}%)")

# Show mismatch breakdown
if len(mismatches) > 0:
    print("\n" + "-"*80)
    print("MISMATCH BREAKDOWN:")
    print("-"*80)
    mismatch_types = mismatches.groupby(['flag', '_expected_flag']).size().reset_index(name='count')
    print(mismatch_types.to_string(index=False))
    
    # Check how many mismatches have same valuenum as valuenum_merged
    print("\n" + "-"*80)
    print("MISMATCH SOURCE ANALYSIS:")
    print("-"*80)
    mismatches['_same_valuenum'] = mismatches['valuenum'] == mismatches['valuenum_merged']
    same_count = mismatches['_same_valuenum'].sum()
    diff_count = len(mismatches) - same_count
    
    print(f"Mismatches where valuenum == valuenum_merged: {same_count:,} ({same_count/len(mismatches)*100:.2f}%)")
    print(f"Mismatches where valuenum != valuenum_merged: {diff_count:,} ({diff_count/len(mismatches)*100:.2f}%)")
    print(f"  → Original data issue: {same_count/len(mismatches)*100:.1f}%")
    print(f"  → Possibly from value extraction: {diff_count/len(mismatches)*100:.1f}%")
    
    print("\n" + "-"*80)
    print("SAMPLE MISMATCHES (first 20):")
    print("-"*80)
    display(mismatches[['hadm_id', 'label', 'value', 'valuenum', 'valuenum_merged', '_ref_low', '_ref_high', 'ref_range', 'qc_flag',
                        'flag', '_expected_flag', '_same_valuenum']].head(20))

# Cleanup helper columns
df.drop(columns=['_val', '_ref_low', '_ref_high', '_expected_flag'], inplace=True)

fix those where  valuenum == valuenum_merged, put np.nan where !=

In [ ]:
# Further fix: For rows with comparison operators (> or <), apply expected flag
# Keep ranges (containing '-') as NaN

# Recompute masks and expected flag
df['_val'] = pd.to_numeric(df['valuenum_merged'], errors='coerce')
df['_ref_low'] = pd.to_numeric(df['ref_range_lower'], errors='coerce')
df['_ref_high'] = pd.to_numeric(df['ref_range_upper'], errors='coerce')

def compute_expected_flag(row):
    val, low, high = row['_val'], row['_ref_low'], row['_ref_high']
    if pd.isna(val) or (pd.isna(low) and pd.isna(high)):
        return np.nan
    if pd.notna(low) and pd.notna(high):
        return 'abnormal' if (val < low or val > high) else 'normal'
    elif pd.notna(low):
        return 'abnormal' if val < low else 'normal'
    elif pd.notna(high):
        return 'abnormal' if val > high else 'normal'
    return np.nan

df['_expected_flag'] = df.apply(compute_expected_flag, axis=1)

# Identify mismatches
mismatch_mask = (df['_expected_flag'].notna()) & (df['flag'] != df['_expected_flag'])

# Check if valuenum == valuenum_merged for mismatches
same_value_mask = df['valuenum'] == df['valuenum_merged']

# Identify comparison values (> or <) vs ranges (-)
has_comparison = df['value'].astype(str).str.contains(r'^[<>]', regex=True, na=False)
has_range = df['value'].astype(str).str.contains(r'^\d+\.?\d*-\d+\.?\d*$', regex=True, na=False)

# Set to NaN where valuenum != valuenum_merged (extraction issue)
nan_mask = mismatch_mask & ~same_value_mask
df.loc[nan_mask, 'flag_corrected'] = np.nan

# Fix comparisons: apply expected flag
comparison_fix_mask = nan_mask & has_comparison
df.loc[comparison_fix_mask, 'flag_corrected'] = df.loc[comparison_fix_mask, '_expected_flag']

print(f"Rows with comparisons (> or <) fixed: {comparison_fix_mask.sum():,}")
print(f"Rows with ranges (-) remaining NaN: {(nan_mask & has_range).sum():,}")

# Cleanup
df.drop(columns=['_val', '_ref_low', '_ref_high', '_expected_flag'], inplace=True)

## to nan: valueuom: has '' , remove; Pos/Neg == +/-; U ??
keep U

In [ ]:
# Handle empty strings and whitespace-only values in valueuom
empty_mask = (df['valueuom'] == '') | (df['valueuom'].astype(str).str.strip() == '')
print(f"  Empty strings '' or whitespace-only: {empty_mask.sum():,} → set to NaN")
df.loc[empty_mask, 'valueuom'] = np.nan

In [ ]:
df['valueuom'] = df['valueuom'].replace({'Pos/Neg': '+/-'})

## Attempt unit normailaztion valueom => valuenum_merged

In [ ]:
df["label"].unique()

In [ ]:
# Format: (label_lowercase, from_unit) : (factor, to_unit)
conversion_map = {
    # Basic Metabolic Panel
    ('glucose', 'mg/dL'): (0.0555, 'mmol/L'),
    ('creatinine', 'mg/dL'): (88.4, 'µmol/L'),
    ('urea nitrogen', 'mg/dL'): (0.357, 'mmol/L'),
    ('bicarbonate', 'mEq/L'): (1.0, 'mmol/L'),
    ('anion gap', 'mEq/L'): (1.0, 'mmol/L'),
    ('calcium, total', 'mg/dL'): (0.25, 'mmol/L'),
    ('magnesium', 'mg/dL'): (0.411, 'mmol/L'),
    ('phosphate', 'mg/dL'): (0.323, 'mmol/L'),
    ('potassium', 'mEq/L'): (1.0, 'mmol/L'),
    ('sodium', 'mEq/L'): (1.0, 'mmol/L'),
    ('triglycerides', 'mg/dL'): (0.0113, 'mmol/L'),
    ('cholesterol, total', 'mg/dL'): (0.0259, 'mmol/L'),
    ('cholesterol, ldl, calculated', 'mg/dL'): (0.0259, 'mmol/L'),
    ('cholesterol, hdl', 'mg/dL'): (0.0259, 'mmol/L'),

    # Hematology
    ('hemoglobin', 'g/dL'): (10.0, 'g/L'),
    ('hematocrit', '%'): (1.0, '%'),
    ('red blood cells', 'M/uL'): (1e6, '/uL'),
    ('white blood cells', 'K/uL'): (1000, '/uL'),
    ('platelet count', 'K/uL'): (1000, '/uL'),
    ('mch', 'pg'): (1.0, 'pg'),
    ('mchc', 'g/dL'): (10.0, 'g/L'),
    ('mcv', 'fL'): (1.0, 'fL'),
    ('rdw', '%'): (1.0, '%'),

    # Coagulation / Blood Gas
    ('ptt', 'sec'): (1.0, 'sec'),
    ('pt', 'sec'): (1.0, 'sec'),
    ('inr(pt)', 'ratio'): (1.0, 'ratio'),
    ('ph', ''): (1.0, ''),
    ('pco2', 'mm Hg'): (0.133, 'kPa'),
    ('po2', 'mm Hg'): (0.133, 'kPa'),
    ('base excess', 'mmol/L'): (1.0, 'mmol/L'),
    ('calculated total co2', 'mmol/L'): (1.0, 'mmol/L'),

    # Enzymes
    ('alanine aminotransferase (alt)', 'U/L'): (1.0, 'U/L'),
    ('asparate aminotransferase (ast)', 'U/L'): (1.0, 'U/L'),
    ('alkaline phosphatase', 'U/L'): (1.0, 'U/L'),
    ('creatine kinase (ck)', 'U/L'): (1.0, 'U/L'),
    ('creatine kinase, mb isoenzyme', 'U/L'): (1.0, 'U/L'),

    # Bilirubin
    ('bilirubin, total', 'mg/dL'): (17.1, 'µmol/L'),
    ('bilirubin', 'mg/dL'): (17.1, 'µmol/L'),

    # Protein / Albumin
    ('albumin', 'g/dL'): (10.0, 'g/L'),
    ('protein, total', 'g/dL'): (10.0, 'g/L'),

    # Lactate / Calcium
    ('lactate', 'mmol/L'): (1.0, 'mmol/L'),
    ('free calcium', 'mg/dL'): (0.25, 'mmol/L'),

    # Blood gases
    ('oxygen saturation', '%'): (1.0, '%'),
}

These include qualitative tests, ratios, percentages, indices, specimen types, or already standardized units:

In [ ]:
non_convertible_labels = [
    'Creatine Kinase (CK)', 'Creatine Kinase, MB Isoenzyme', 'Troponin T', 'INR(PT)',
    'PT', 'PTT', '% Hemoglobin A1c', 'eAG', 'Anion Gap', 'Cholesterol Ratio (Total/HDL)',
    'Specimen Type', 'EDTA Hold', 'Red Top Hold', 'Intubated', 'Ventilator', 'PEEP',
    'Ventilation Rate', 'Required O2', 'H', 'L', 'I', 'Uhold', 'Other', 'Problem Specimen',
    'Voided Specimen', 'Urine Specimen Type', 'Blue Top Hold', 'Green Top Hold (plasma)',
    'Gray Top Hold (plasma)', 'Light Green Top Hold', 'Red Top Hold', 'Plasma', 'NRBC',
    'WBC Clumps', 'Other Cells', 'Other Cell', 'NonSquamous Epithelial Cell',
    'Non-squamous Epithelial Cells', 'Urobilinogen', 'Urine Appearance', 'Urine Color',
    'Urine Mucous', 'Bacteria', 'Blood', 'Epithelial Cells', 'Hyaline Casts', 'Ketone',
    'Leukocytes', 'Nitrite', 'Yeast', 'Amorphous Crystals', 'Voided Specimen',
    'Macrocytes', 'Microcytes', 'Schistocytes', 'Teardrop Cells', 'Echinocytes',
    'Spherocytes', 'Ovalocytes', 'Bite Cells', 'Target Cells', 'Elliptocytes', 'Pencil Cells',
    'Sickle Cells', 'Triple Phosphate Crystals', 'Waxy Casts', 'Granular Casts',
    'Cellular Cast', 'RBC Casts', 'Protein Electrophoresis', 'Immunofixation', 'CK-MB Index',
    'Free Kappa', 'Free Lambda', 'Free Kappa/Free Lambda Ratio', 'Immature Granulocytes',
    'CD markers', 'Anti-Nuclear Antibody', 'Hepatitis B Surface Antibody', 'Hepatitis C Virus Antibody',
    'HIV Screen', 'Influenza A by PCR', 'Influenza B by PCR', 'G6PD, Qualitative',
    'Calculated Thyroxine (T4) Index', 'Uptake Ratio', 'Calculated TBG', 'EtOH',
    'Specimen-related holds', 'Ventilation/oxygen settings'
]


In [ ]:
for (analyte, from_unit), (factor, to_unit) in conversion_map.items():
    mask = (df['label'].str.lower() == analyte) & (df['valueuom'] == from_unit)
    df.loc[mask, 'valuenum_merged'] *= factor
    df.loc[mask, 'valueuom'] = to_unit

## Handle missing values

In [ ]:
# print sum of all missing values per column
for col in df.columns:
    missing_count = df[col].isna().sum()
    if missing_count > 0:
        print(f"Column '{col}': {missing_count} missing values")

## Do scatterpltos / distr / etc to check for dataqualtiy

### Do correlation matrix

## Save

In [ ]:
df.to_csv(f"{DATA_DIR}/{DATASETS[name].replace('.csv', '_cleaned.csv')}", index=False)

In [ ]:
# load already cleaned to skip first steps
#df = pd.read_csv(f"{DATA_DIR}/heart_diagnoses_1_cleaned.csv")

In [ ]:
df.info()

## add subject id from df1, df3, and df4

## Create features and slim version

In [ ]:
df_slim = df.copy()

In [ ]:
df_slim.columns

In [ ]:
df_slim[df_slim.columns[:10]]

In [ ]:
df_slim[df_slim.columns[10:]]

### Create features
- var for exams
- var for charttime?
- var for dod (is_dead?), anchor_year?

### Corr

In [ ]:
print("="*80)
print("CORRELATION MATRIX ANALYSIS")
print("="*80)

# Select numeric columns only
numeric_cols = df_slim.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumeric columns found: {numeric_cols}")
print(f"Total numeric columns: {len(numeric_cols)}\n")

if len(numeric_cols) > 1:
    # Calculate correlation matrix
    correlation_matrix = df_slim[numeric_cols].corr()
    
    print("="*80)
    print("CORRELATION MATRIX")
    print("="*80)
    print(correlation_matrix.round(3))
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Plot 1: Full correlation heatmap
    ax1 = axes[0]
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0,
                square=True,
                linewidths=0.5,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=ax1)
    ax1.set_title('Full Correlation Matrix Heatmap', fontsize=14, fontweight='bold')
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    plt.setp(ax1.get_yticklabels(), rotation=0)
    
    # Plot 2: Mask for upper triangle (cleaner view)
    ax2 = axes[1]
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0,
                square=True,
                linewidths=0.5,
                mask=mask,
                cbar_kws={'label': 'Correlation Coefficient'},
                ax=ax2)
    ax2.set_title('Lower Triangle Correlation Matrix (Unique Pairs)', fontsize=14, fontweight='bold')
    plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
    plt.setp(ax2.get_yticklabels(), rotation=0)
    
    plt.tight_layout()
    plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Extract strong correlations (> 0.5 or < -0.5, excluding diagonal)
    print("\n" + "="*80)
    print("STRONG CORRELATIONS (|r| > 0.5, excluding self-correlations)")
    print("="*80)
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:
                strong_corr.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlation': corr_value,
                    'Strength': 'Strong Positive' if corr_value > 0 else 'Strong Negative'
                })
    
    if strong_corr:
        strong_corr_df = pd.DataFrame(strong_corr).sort_values('Correlation', key=abs, ascending=False)
        print(f"\nFound {len(strong_corr)} strong correlations:\n")
        print(strong_corr_df.to_string(index=False))
    else:
        print("\nNo correlations with |r| > 0.5 found")
    
    # Moderate correlations (0.3 to 0.5)
    print("\n" + "="*80)
    print("MODERATE CORRELATIONS (0.3 < |r| ≤ 0.5)")
    print("="*80)
    
    moderate_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if 0.3 < abs(corr_value) <= 0.5:
                moderate_corr.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlation': corr_value,
                    'Strength': 'Moderate Positive' if corr_value > 0 else 'Moderate Negative'
                })
    
    if moderate_corr:
        moderate_corr_df = pd.DataFrame(moderate_corr).sort_values('Correlation', key=abs, ascending=False)
        print(f"\nFound {len(moderate_corr)} moderate correlations:\n")
        print(moderate_corr_df.to_string(index=False))
    else:
        print("\nNo moderate correlations found (0.3 < |r| ≤ 0.5)")
    
    # Summary statistics
    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    
    # Get correlation values (excluding diagonal)
    corr_values = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_values.append(correlation_matrix.iloc[i, j])
    
    corr_values = np.array(corr_values)
    print(f"\nMean correlation: {corr_values.mean():.3f}")
    print(f"Median correlation: {np.median(corr_values):.3f}")
    print(f"Std Dev: {corr_values.std():.3f}")
    print(f"Min: {corr_values.min():.3f}")
    print(f"Max: {corr_values.max():.3f}")
    
    # Distribution of correlations
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.hist(corr_values, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(corr_values.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {corr_values.mean():.3f}')
    ax.axvline(np.median(corr_values), color='green', linestyle='--', linewidth=2, label=f'Median: {np.median(corr_values):.3f}')
    ax.set_xlabel('Correlation Coefficient', fontsize=12, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax.set_title('Distribution of Correlation Coefficients', fontsize=13, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig('correlation_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print("⚠️ Not enough numeric columns for correlation analysis")
    print(f"Found only {len(numeric_cols)} numeric column(s)")

### Dropunused

In [ ]:
df_slim.info()

In [ ]:
df_slim.columns, len(df_slim.columns)

In [ ]:
# DROP: 'note_id', 'note_type', 'note_seq', 'subject_id_dx', 'storetime', 'HPI', 'physical_exam', 'chief_complaint', 'invasions', 'X-ray', 'CT', 'Ultrasound', 'CATH', 'MRI', 'reports', 'subject_id_dx',  'anchor_year', 'dod', 'has_x-ray', 'has_ct', 'has_ultrasound', 'has_cath', 'has_ecg', 'has_mri', 'age_group'

# keep 'subject_id', 'hadm_id' , icd_code, 'ECG' (for later task 3), charttime
df_slim.drop(columns=[ 'note_id', 'note_type', 'note_seq', 'subject_id_dx', 'storetime', 'HPI', 'physical_exam', 'chief_complaint', 'invasions', 'X-ray', 'CT', 'Ultrasound', 'CATH', 'MRI', 'reports', 'subject_id_dx',  'anchor_year', 'dod', 'has_x-ray', 'has_ct', 'has_ultrasound', 'has_cath', 'has_ecg', 'has_mri', 'age_group'], inplace=True)
df_slim.columns, len(df_slim.columns)

In [ ]:
df_slim[['exam_type', 'exam_type_label']].tail()

In [ ]:
df_slim['has_cardiac_exam'].value_counts(), df_slim['has_radiology_exam'].value_counts()

In [ ]:
df_slim['exam_type_label'].value_counts()

In [ ]:
df_slim['is_dead'].value_counts()

In [ ]:
df_slim.drop(columns=['long_title', 'has_radiology_exam' ], inplace=True)
df_slim.columns, len(df_slim.columns)


## Save SLim

In [ ]:
df_slim.to_csv(f"{DATA_DIR}/heart_diagnoses_1_cleaned_slim.csv", index=False)